In [1]:
import numpy as np
import pandas as pd
import sklearn

In [2]:
pokemon = pd.read_csv("pokemon.csv")
combats = pd.read_csv("combats.csv")
X_combats = combats[combats.columns[:2]]
y = 1*(combats.values[:,0]==combats.values[:,2])

In [3]:
pokemon = pokemon[['#','Type 1', 'Type 2', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']]

In [4]:
print(pokemon.head())
print(X_combats.head())

   # Type 1  Type 2  HP  Attack  Defense  Sp. Atk  Sp. Def  Speed
0  1  Grass  Poison  45      49       49       65       65     45
1  2  Grass  Poison  60      62       63       80       80     60
2  3  Grass  Poison  80      82       83      100      100     80
3  4  Grass  Poison  80     100      123      122      120     80
4  5   Fire     NaN  39      52       43       60       50     65
   First_pokemon  Second_pokemon
0            266             298
1            702             701
2            191             668
3            237             683
4            151             231


In [5]:
m1 = X_combats.merge(pokemon, left_on = 'First_pokemon', right_on = '#', how = 'left')
X = m1.merge(pokemon, left_on = 'Second_pokemon', right_on = '#', how = 'left')
X = X[['Type 1_x', 'Type 2_x','HP_x', 'Attack_x', 'Defense_x', 'Sp. Atk_x', 'Sp. Def_x', 'Speed_x', 'Type 1_y', 'Type 2_y', 'HP_y', 'Attack_y', 'Defense_y',
       'Sp. Atk_y', 'Sp. Def_y', 'Speed_y']]

In [6]:
X = X.fillna('Notype')

In [7]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
type_encoder = LabelEncoder()
type_onehot = OneHotEncoder()
typelist = X[['Type 1_x','Type 2_x','Type 1_y', 'Type 2_y']].values.flatten()
type_nums = type_encoder.fit_transform(typelist)
onehottypes = type_onehot.fit_transform(type_nums.reshape(-1,1))

In [8]:
type_frames = {}
for name in ['Type 1_x','Type 2_x','Type 1_y', 'Type 2_y']:
    encoded = type_onehot.transform(type_encoder.transform(X[name]).reshape(-1,1)).toarray()
    type_frames[name] = pd.DataFrame(encoded, columns = [name + ' ' + elem for elem in type_encoder.inverse_transform(range(19))])

C:\Users\Mateusz\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Mateusz\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Mateusz\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
C:\Users\Mateusz\Anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning Fals

In [9]:
types_encoded = pd.concat([type_frames[elem] for elem in ['Type 1_x','Type 2_x','Type 1_y', 'Type 2_y']], axis = 1)

In [10]:
X = X[['HP_x', 'Attack_x', 'Defense_x', 'Sp. Atk_x', 'Sp. Def_x', 'Speed_x', 'HP_y', 'Attack_y', 'Defense_y', 'Sp. Atk_y', 'Sp. Def_y', 'Speed_y']]
X = pd.concat([X,types_encoded], axis = 1)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=0)
clf.fit(X_train,y_train)
res = clf.predict(X_test)
np.mean(res == y_test)

0.8926

In [13]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
clf.fit(X_train,y_train)
res = clf.predict(X_test)
np.mean(res == y_test)

0.9523

In [18]:
from sklearn.tree import ExtraTreeClassifier
clf = ExtraTreeClassifier(random_state=0)
clf.fit(X_train,y_train)
res = clf.predict(X_test)
np.mean(res == y_test)

0.8098